In [ ]:
from flask import Flask, request, jsonify
import pymysql
from konlpy.tag import Okt
from numpy import dot
from numpy.linalg import norm
import numpy as np
import pandas as pd 
import openpyxl
import json


app = Flask(__name__)

def cos_sim(a, b):
    norm_a = np.linalg.norm(a)
    norm_b = np.linalg.norm(b)
    if norm_a == 0 or norm_b == 0:
        return 0  
    return np.dot(a, b) / (norm_a * norm_b)


@app.route('/api/contents/lastProject.ipynb/execute', methods=['POST'])
def execute_code():
    
    request_data = request.json
    
   
    login_id_val = request_data.get('login_id')
    
   
    conn = pymysql.connect(host='localhost', user='root',
                       password='a190256!', db='saving_service', charset='utf8mb4')
    cursor = conn.cursor()
    cursor.execute('''select * from member where login_id = %s''', (login_id_val,))

    rows = cursor.fetchall()

    
    list_data = []
    for i in rows:
     my_list = list(i)
     if my_list[9] == "대학생":
      list_data.append(0)
     else:
      list_data.append(1)

     if my_list[4] == "예":
      list_data.append(0)
     else :
      list_data.append(1)

     if my_list[2] == "예":
       list_data.append(0)
     else:
       list_data.append(1)

     if my_list[3] == "높은금리":
      list_data.append(0)
     elif my_list[3] == "해당 금융사에 대한 평판":
      list_data.append(1)
     elif my_list[3] == "우대조건":
      list_data.append(2)
     elif my_list[3] == "편의성" :
      list_data.append(3)

     if my_list[7] == "6개월":
      list_data.append(0)
     elif my_list[7] == "12개월":
      list_data.append(1)
     elif my_list[7] == "24개월":
      list_data.append(2)
     elif my_list[7] == "36개월":
      list_data.append(3)
    df = pd.read_excel('digit.xlsx', sheet_name = 0, index_col = 0)
    
    df.columns = ['상태', '관심','사용', '가진상품', '중요', '기간']
    test_df =df.drop('가진상품' , axis = 1)
    test_df = test_df.drop(df.index[21], axis = 0)
    test_df = test_df.drop(df.index[63], axis = 0)
    list_from_df = test_df.values.tolist()

    max_index = 0
    count = -1
    max_cos_similarity = 0
    for i in list_from_df:
        count +=1
        storage = cos_sim(list_data, i)
        if storage > max_cos_similarity:
          max_cos_similarity = storage
          max_index = count
    max_index = str(max_index) 
    cursor.execute('''select commodity from google_form_member where member_id = %s''', (max_index,))
    result = cursor.fetchone()
    if result:
        
        commodity_string = result[0]  
        commodity_list = commodity_string.split(',')  
        for commodity in commodity_list:
          sql = "UPDATE member SET recommendation = %s WHERE login_id = %s"
          cursor.execute(sql,(commodity,login_id_val)) 
        conn.commit()  
    else:
        print("No result found.")    
    cursor.close()
    conn.close()

    result = f"{login_id_val} 님과 비슷한 사람이 가진 상품: {commodity_string}"
    
 
    return jsonify(result=result)
    
    

if __name__ == '__main__':
    app.run(port=8888)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:8888
Press CTRL+C to quit
